In [71]:
import pandas as pd
import numpy as np
import pandas as pd
from flask import Flask, render_template, request
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.metrics.pairwise import linear_kernel, cosine_similarity
import json
import bs4 as bs
import urllib.request
import pickle
import requests
import Levenshtein



In [72]:
def similarity_tags():
    movie_df_ex = pd.read_csv('tags_recom.csv')
    movie_df_ex=movie_df_ex.drop(['level_0', 'index'], axis=1)
    movie_df_ex['title']=movie_df_ex['title'].str.lower()
    tf = TfidfVectorizer(analyzer='word',ngram_range=(1, 2),min_df=0, stop_words='english')
    c  = CountVectorizer()
    tfidf_mtx = c.fit_transform(movie_df_ex['description'].astype('U').values)
    cosine_sim = cosine_similarity(tfidf_mtx, tfidf_mtx)
    movie_df_ex = movie_df_ex.reset_index()
    titles = movie_df_ex['title']
    indices = pd.Series(movie_df_ex.index, index=movie_df_ex['title'])
    return movie_df_ex,indices,cosine_sim,titles
    
def recommendations_tags(title):    
#     m = title.lower()
    movie_df_ex,indices,cosine_sim,titles = similarity_tags()
#     print(movie_df_ex['title'].lower())
    if title not in movie_df_ex['title'].str.lower().unique():
        return('Sorry! The movie you requested is not in our database. Please check the spelling or try with some other movies')
    else:
        
        idx = indices[title]
        sim_scores = list(enumerate(cosine_sim[idx]))
        sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
        sim_scores = sim_scores[1:11]
        movie_indices = [i[0] for i in sim_scores]
        i = titles.iloc[movie_indices]
#         print(titles.iloc[movie_indices])
        l = []
        for i in titles.iloc[movie_indices]:
            l.append(i)
    return l

In [73]:
recommendations_tags('the lion king')

['raising helen',
 'hands in the air',
 'up',
 'abduction',
 'precious',
 "the lion king 2: simba's pride",
 'y tu mamá también',
 'all the pretty horses',
 'unstrung heroes',
 'along came polly']

In [74]:
def similarity_metadata():
    smovie_df = pd.read_csv('content_recom.csv')
#     smovie_df=smovie_df.drop(['level_0', 'index'], axis=1)
    smovie_df['title']=smovie_df['title'].str.lower()
    count = CountVectorizer(analyzer='word',ngram_range=(1, 2),min_df=0, stop_words='english')
    count_matrix = count.fit_transform(smovie_df['soup'])
    cosine_sim = cosine_similarity(count_matrix, count_matrix)
    smovie_df = smovie_df.reset_index()
    titles = smovie_df['title']
    indices = pd.Series(smovie_df.index, index=smovie_df['title'])
    return smovie_df,indices,cosine_sim,titles

In [75]:
def weighted_rating(x):
    v = x['vote_count']
    R = x['vote_average']
    return (v/(v+434.0) * R) + (434.0/(434.0+v) * 5.244896612406511)

In [76]:
def recommendations_content(title):    
    m = title.lower()
    smovie_df,indices,cosine_sim,titles = similarity_metadata()
    if title not in smovie_df['title'].str.lower().unique():
        return('Sorry! The movie you requested is not in our database. Please check the spelling or try with some other movies')
    else:
        
        idx = indices[title]
        sim_scores = list(enumerate(cosine_sim[idx]))
        sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
        sim_scores = sim_scores[1:26]
        movie_indices = [i[0] for i in sim_scores]

        movies = smovie_df.iloc[movie_indices][['title', 'vote_count', 'vote_average', 'year']]
        vote_counts = movies[movies['vote_count'].notnull()]['vote_count'].astype('int')
        vote_averages = movies[movies['vote_average'].notnull()]['vote_average'].astype('int')
        C = vote_averages.mean()
        m = vote_counts.quantile(0.60)
        qualified = movies[(movies['vote_count'] >= m) & (movies['vote_count'].notnull()) & (movies['vote_average'].notnull())]
        qualified['vote_count'] = qualified['vote_count'].astype('int')
        qualified['vote_average'] = qualified['vote_average'].astype('int')
        qualified['wr'] = qualified.apply(weighted_rating, axis=1)
        qualified = qualified.sort_values('wr', ascending=False).head(10)
#         print(titles.iloc[movie_indices])
#         print(qualified)
        l = []
        for i in qualified['title'].head(10):
            l.append(i)
    return l

In [77]:
recommendations_content('liar liar')

/Users/sudeepsawant/miniforge3/envs/mlp/lib/python3.8/site-packages/pandas/core/frame.py:3607: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._set_item(key, value)


['patch adams',
 'bruce almighty',
 'cloud atlas',
 'ace ventura: pet detective',
 'that thing you do!',
 'dragonfly',
 'larry crowne',
 'failure to launch',
 'the cobbler',
 'evan almighty']

In [78]:
import scipy.stats

In [79]:
x = np.arange(10, 20)
y = np.array([2, 1, 4, 5, 8, 12, 18, 25, 96, 48])
scipy.stats.pearsonr(x, y) 

(0.7586402890911867, 0.01096434130168085)

In [80]:
from sklearn.metrics.pairwise import sigmoid_kernel
from sklearn.feature_extraction.text import TfidfVectorizer
def similarity_metadata_sigmoid():
    smovie_df = pd.read_csv('content_recom.csv')
#     smovie_df=smovie_df.drop(['level_0', 'index'], axis=1)
    smovie_df['title']=smovie_df['title'].str.lower()
    count = TfidfVectorizer(strip_accents='unicode',analyzer='word',ngram_range=(1, 3),min_df=0, stop_words='english')
    count_matrix = count.fit_transform(smovie_df['soup'])
#     print(count_matrix[0])
    cosine_sim = sigmoid_kernel(count_matrix, count_matrix)
    smovie_df = smovie_df.reset_index()
    titles = smovie_df['title']
    indices = pd.Series(smovie_df.index, index=smovie_df['title'])
    return smovie_df,indices,cosine_sim,titles

In [81]:
def recommendations_content_sigmoid(title):    
    m = title.lower()
    smovie_df,indices,cosine_sim,titles = similarity_metadata_sigmoid()
    if title not in smovie_df['title'].str.lower().unique():
        return('Sorry! The movie you requested is not in our database. Please check the spelling or try with some other movies')
    else:
        
        idx = indices[title]
        sim_scores = list(enumerate(cosine_sim[idx]))
        sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
        sim_scores = sim_scores[1:26]
        movie_indices = [i[0] for i in sim_scores]

        movies = smovie_df.iloc[movie_indices][['title', 'vote_count', 'vote_average', 'year']]
        vote_counts = movies[movies['vote_count'].notnull()]['vote_count'].astype('int')
        vote_averages = movies[movies['vote_average'].notnull()]['vote_average'].astype('int')
        C = vote_averages.mean()
        m = vote_counts.quantile(0.60)
        qualified = movies[(movies['vote_count'] >= m) & (movies['vote_count'].notnull()) & (movies['vote_average'].notnull())]
        qualified['vote_count'] = qualified['vote_count'].astype('int')
        qualified['vote_average'] = qualified['vote_average'].astype('int')
        qualified['wr'] = qualified.apply(weighted_rating, axis=1)
        qualified = qualified.sort_values('wr', ascending=False).head(10)
#         print(titles.iloc[movie_indices])
#         print(qualified)
        l = []
        for i in qualified['title'].head(10):
            l.append(i)
    return l

In [82]:
recommendations_content_sigmoid ('cars')

['toy story',
 'toy story 2',
 "a bug's life",
 '2 fast 2 furious',
 'the fast and the furious: tokyo drift',
 'gone in sixty seconds',
 'vacation',
 'death race',
 'little fockers',
 'cars 2']

In [83]:
recommendations_content('cars')

['toy story',
 'toy story 2',
 "ferris bueller's day off",
 'an american werewolf in london',
 'planes, trains and automobiles',
 "a bug's life",
 '2 fast 2 furious',
 'the fast and the furious: tokyo drift',
 'the rescuers',
 'cars 2']

In [84]:
def similarity_tags_sigmoid():
    movie_df_ex = pd.read_csv('tags_recom.csv')
    movie_df_ex=movie_df_ex.drop(['level_0', 'index'], axis=1)
    movie_df_ex['title']=movie_df_ex['title'].str.lower()
    tf = TfidfVectorizer(analyzer='word',ngram_range=(1, 3),min_df=0, stop_words='english')
    
    tfidf_mtx = tf.fit_transform(movie_df_ex['description'].astype('U').values)
    cosine_sim = sigmoid_kernel(tfidf_mtx, tfidf_mtx)
    movie_df_ex = movie_df_ex.reset_index()
    titles = movie_df_ex['title']
    indices = pd.Series(movie_df_ex.index, index=movie_df_ex['title'])
    return movie_df_ex,indices,cosine_sim,titles
    
def recommendations_tags_sigmoid(title):    
#     m = title.lower()
    movie_df_ex,indices,cosine_sim,titles = similarity_tags_sigmoid()
#     print(movie_df_ex['title'].lower())
    if title not in movie_df_ex['title'].str.lower().unique():
        return('Sorry! The movie you requested is not in our database. Please check the spelling or try with some other movies')
    else:
        
        idx = indices[title]
        sim_scores = list(enumerate(cosine_sim[idx]))
        sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
        sim_scores = sim_scores[1:11]
        movie_indices = [i[0] for i in sim_scores]
        i = titles.iloc[movie_indices]
#         print(titles.iloc[movie_indices])
        l = []
        for i in titles.iloc[movie_indices]:
            l.append(i)
    return l

In [85]:
recommendations_tags_sigmoid('liar liar')

['all i want for christmas',
 '16 wishes',
 'a goofy movie',
 'an extremely goofy movie',
 'truly madly deeply',
 'go fish',
 'the secret life of pets',
 'heartbreakers',
 'bent',
 'kissing a fool']

In [86]:
recommendations_tags('liar liar')

['a goofy movie',
 '16 wishes',
 'an extremely goofy movie',
 'go fish',
 'wish upon a star',
 'the secret life of pets',
 'october sky',
 'heartbreakers',
 'all i want for christmas',
 'rushmore']

In [87]:
from scipy import sparse
def get_similar(movie_name,rating,corrMatrix):
    similar_ratings = corrMatrix[movie_name]*(rating-2.5)
    similar_ratings = similar_ratings.sort_values(ascending=False)
    #print(type(similar_ratings))
    
    return similar_ratings

def colab(movie_name,rating):
    userRatings = pd.read_csv('ratings.csv')
    corrMatrix = userRatings.corr(method='pearson')
#     corrMatrix.head(100)
    similar_movies = pd.DataFrame()
    


    return get_similar(movie_name,rating,corrMatrix).head(10) 